@[TOC]

# 1.关于Variable和Tensor

旧版本的Pytorch中，Variable是对Tensor的一个封装；在Pytorch大于v0.4的版本后，Varible和Tensor合并了，意味着Tensor可以像旧版本的Variable那样运行，当然新版本中Variable封装仍旧可以用，但是对Varieble操作返回的将是一个Tensor。

In [1]:
import torch
from torch.autograd import Variable

In [3]:
torch.__version__

'1.3.1'

In [24]:
x = torch.Tensor(3, 4)
x

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [25]:
type(x)

torch.Tensor

In [32]:
x.requires_grad  # 默认为False

False

In [26]:
y = Variable(x)
print(y)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [27]:
type(y)

torch.Tensor

## 1.1 torch.Tensor和torch.tensor有区别

#### (1) 类
在PyTorch 中，`torch.Tensor`是主要的tensor类，所有的tensor都是`torch.Tensor`的实例。
`torch.Tensor`是`torch.FloatTensor`的别名。

而torch.tensor是一个函数，返回的是一个tensor，在PyTorch官方文档中，描述如下：
```python
torch.tensor(data, dtype=None, device=None, requires_grad=False) → Tensor
Constructs a tensor with data.
```

#### (2) dtype类型
所以需要注意的一点是：

- torch.Tensor（data）是将输入的data转化`torch.FloatTensor`
- torch.tensor(data):(当你未指定`dtype`的类型时)将data转化为`torch.FloatTensor`、`torch.LongTensor`、`torch.DoubleTensor`等类型，转化类型依据于data的类型或者`dtype`的值


#### (3)创建空tensor
使用如下语句：tensor_without_data = torch.Tensor()可以创建一个空的FloatTensor，而当你使用tensor_without_data = torch.tensor()时候则会报错

当你想要创建一个空的tensor时候，可以使用如下的方法：

- tensor_without_data = torch.Tensor() # tensor([ ])
- tensor_without_data = torch.tensor(( )) # tensor([ ])
- tensor_without_data = torch.empty([ ]) # tensor(0.)

所以`torch.Tensor`应该说是同时具有`torch.tensor`和`torch.empty`的功能，但是使用`torch.Tensor`可能会使你的代码confusing，所以最好还是使用`torch.tensor`和`torch.empty`，而不是`torch.Tensor`。

In [28]:
x == y

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

In [33]:
import torch as t
from torch.autograd import Variable
 
a = t.ones(3,requires_grad=True)
print(type(a))
#输出：<class 'torch.Tensor'>
 
a=Variable(a)
print(type(a))
#输出仍旧是：<class 'torch.Tensor'>
 
print(a.volatile)
#输出：__main__:1: UserWarning: volatile was removed (Variable.volatile is always False)
a.volatile=True
print(a.volatile)
#输出：__main__:1: UserWarning: volatile was removed (Variable.volatile is always False)
#现版本pytorch中移除了volatile这个属性，即volatile总是false

<class 'torch.Tensor'>
<class 'torch.Tensor'>
False
False


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: volatile was removed (Variable.volatile is always False)
  if sys.path[0] == '':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed (Variable.volatile is always False)
  from ipykernel import kernelapp as app


# 2.叶子节点leaf

对于那些不是任何函数(Function)的输出，由用户创建的节点称为叶子节点，叶子节点的grad_fn为None。

**grad_fn 指向Function对象，用于反向传播的梯度计算之用**

In [30]:
import torch as t
a = t.ones(3,requires_grad=True)
b = t.rand(3,requires_grad=True)
print(a, a.is_leaf)  # 输出：(tensor([1., 1., 1.], requires_grad=True), True)

print(b)  # 输出：tensor([0.4254, 0.8763, 0.5901], requires_grad=True)

 
c = a*b
print(c.is_leaf)  # 输出：False.说明c不是叶子节点

print(a.grad_fn)  # 输出：None.叶子节点的grad_fn为None.

print(c.grad_fn)  # 输出：<MulBackward0 object at 0x7fa45c406278> 

tensor([1., 1., 1.], requires_grad=True) True
tensor([0.1666, 0.6333, 0.2296], requires_grad=True)
False
None


# 3. a为Tensor变量. 有很多方法和属性 可以用help(a)查看

In [54]:
print(help(a.backward))

Help on method backward in module torch.tensor:

backward(gradient=None, retain_graph=None, create_graph=False) method of torch.Tensor instance
    Computes the gradient of current tensor w.r.t. graph leaves.
    
    The graph is differentiated using the chain rule. If the tensor is
    non-scalar (i.e. its data has more than one element) and requires
    gradient, the function additionally requires specifying ``gradient``.
    It should be a tensor of matching type and location, that contains
    the gradient of the differentiated function w.r.t. ``self``.
    
    This function accumulates gradients in the leaves - you might need to
    zero them before calling it.
    
    Arguments:
        gradient (Tensor or None): Gradient w.r.t. the
            tensor. If it is a tensor, it will be automatically converted
            to a Tensor that does not require grad unless ``create_graph`` is True.
            None values can be specified for scalar Tensors or ones that
            don't

In [41]:
help(a.expand)

Help on built-in function expand:

expand(...) method of torch.Tensor instance
    expand(*sizes) -> Tensor
    
    Returns a new view of the :attr:`self` tensor with singleton dimensions expanded
    to a larger size.
    
    Passing -1 as the size for a dimension means not changing the size of
    that dimension.
    
    Tensor can be also expanded to a larger number of dimensions, and the
    new ones will be appended at the front. For the new dimensions, the
    size cannot be set to -1.
    
    Expanding a tensor does not allocate new memory, but only creates a
    new view on the existing tensor where a dimension of size one is
    expanded to a larger size by setting the ``stride`` to 0. Any dimension
    of size 1 can be expanded to an arbitrary value without allocating new
    memory.
    
    Args:
        *sizes (torch.Size or int...): the desired expanded size
    
    .. warning::
    
        More than one element of an expanded tensor may refer to a single
        me

In [65]:
help(a.grad_fn)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [48]:
x = torch.tensor([[1], [2], [3]])
x.size()

torch.Size([3, 1])

In [49]:
y = x.expand(3, 4)
y

tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

In [50]:
y.size()

torch.Size([3, 4])

In [52]:
z = x.expand(-1, 4)
z

tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

In [53]:
z.size()

torch.Size([3, 4])

In [56]:
y == z  # 两者相等

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

# 4. 求导 - autograd操作

首先Tensor是默认不需要求导的，即requires_grad默认为False。

In [58]:
d = torch.ones(3)
d.requires_grad
#输出：False.Tensor默认不需要求导

False

**如果某一个节点requires_grad被设置为True，那么所有依赖它的节点requires_grad都为True。**

In [62]:
import torch
a = torch.ones(3)
b = torch.ones(3,requires_grad=True)
print(b.requires_grad)  # 输出：True, 注意 a.requires_grad 仍然为False

c = a + b
print(c.requires_grad)  # 输出：True.虽然c没有指定需要求导，然是c依赖于b，而b需要求导，所以c.requires_grad=True

True
True


**并不是只有scaler才能进行backward操作，矩阵和向量也可以，只不过backward()中要添加对应维度的参数。**

- retain_graph=True是为了保存中间缓存，否则再次backward的时候会报错

In [72]:
import torch
 
a = torch.ones(3,requires_grad=True)
b = torch.rand(3,requires_grad=True)

# 输出：(tensor([1., 1., 1.], requires_grad=True), 
# tensor([0.9373, 0.0556, 0.6426], requires_grad=True))
print('1:', a, b)

c = a*b
print('2:', c)  # 输出：tensor([0.9373, 0.0556, 0.6426], grad_fn=<MulBackward0>)


# 报错：RuntimeError: grad can be implicitly created only for scalar outputs
# 只有数值scalar才能进行backward操作
# print('3:', c.backward(retain_graph=True))  # 报错

d = c.sum()
print('4:', d.backward(retain_graph=True))
# retain_graph=True是为了保存中间缓存，否则再次backward的时候会报错

print('5:', a.grad)  # a的导数是b
#输出：tensor([0.9373, 0.0556, 0.6426])

print('6:', b.grad)  # b的导数是a
#输出：tensor([1., 1., 1.])
#backward后a和b的grad产生了数值

e = c.sum()
e.backward(retain_graph=True)
print('7:', b.grad)
#输出：tensor([2., 2., 2.]).b的grad进行了两次backward后进行了累加.

f = c.sum()
print('8:', b.grad)
#输出：tensor([2., 2., 2.])
#只进行计算不backward，梯度不更新

1: tensor([1., 1., 1.], requires_grad=True) tensor([0.6488, 0.8633, 0.6592], requires_grad=True)
2: tensor([0.6488, 0.8633, 0.6592], grad_fn=<MulBackward0>)
4: None
5: tensor([0.6488, 0.8633, 0.6592])
6: tensor([1., 1., 1.])
7: tensor([2., 2., 2.])
8: tensor([2., 2., 2.])


# 5. Tensor.data和Tensor.detach()

如过tensor的数值需要参与计算又不想参与到计算图的更新中，计算的时候可以用tensor.data，这样既能利用tensor的数值，又不会更新梯度。

In [90]:
import torch
 
a = torch.ones(3,4,requires_grad=True)
b = torch.rand(3,4,requires_grad=True)
 
print(a.data.requires_grad, a.requires_grad)  # False True # 因为a.data独立于计算图之外

c = a.data * b.data
d = c.sum()
print(d.backward)
# print(d.backward())  # 报错
# 输出：RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
# 因为独立于计算图之外，requires_grad = False所以不能backward()

False True
<bound method Tensor.backward of tensor(4.9418)>


In [92]:
c = a * b
d = c.sum()
print(d.backward)  # 注意这个d中有grad_fn属性

<bound method Tensor.backward of tensor(4.9418, grad_fn=<SumBackward0>)>


**当tensor.data被修改的时候，tensor也会同步的被修改，此时用该tensor进行计算并backward的时候梯度的值就不再准确了，因为tensor已经被修改了！**

In [95]:
import torch
 
a = torch.ones(3,4,requires_grad=True)
b = torch.rand(3,4,requires_grad=True)
c = a*b
d = c.sum()
print('1:', a.data.sigmoid_())
#输出：tensor([[0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311]])
#虽然对a.data进行sigmoid操作，但是a的值已经被修改了.

d.backward()
print('2:', b.grad)
#输出：tensor([[0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311]])
#b的grad不准了，本来应该都是1！

1: tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]])
2: tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]])


**为了避免因为对tensor.data修改导致grad变化的情况，可以利用tensor.detach，同样可以保证tensor不参与到计算图当中，
但是当tensor的值被改变的时候，再进行backward就会 __报错__ 而不会有先前的因为tensor的值被改变而导致不准的情况了。**

In [97]:
import torch
 
a = torch.ones(3,4,requires_grad=True)
b = torch.rand(3,4,requires_grad=True)
c = a * b
d = c.sum()
a_ = a.detach()
a_.sigmoid_()
print(a)
print(a_)
#输出：tensor([[0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311],
#        [0.7311, 0.7311, 0.7311, 0.7311]], requires_grad=True)
#a的值已经发生了改变
# d.backward()  # 报错
#报错：RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation
#因为a的值被修改了，所以不能再进行backward

tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]], requires_grad=True)
tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]])


**推荐用tensor.detach的方式而不是tensor.data的方式，因为这样更保险！**

# 6.autograd.grad和hook
在计算的时候有时候我们可能会用到非叶节点的grad，但是非叶节点的grad在backward之后就会被自动清空：

In [102]:
import torch
 
a = torch.ones(3,4,requires_grad=True)
b = torch.rand(3,4,requires_grad=True)
c = a*b
d = c.sum()
d.backward()
print(a.grad)
#输出：tensor([[0.3114, 0.3017, 0.8461, 0.6899],
#        [0.3878, 0.8712, 0.2406, 0.7396],
#        [0.6369, 0.0907, 0.4984, 0.5058]])

print(c.grad)
#输出：None
#c为非叶子节点，计算后被清空

tensor([[0.9379, 0.0388, 0.9329, 0.9576],
        [0.7599, 0.5928, 0.9593, 0.8715],
        [0.4793, 0.1550, 0.8241, 0.1743]])
None


**可以用autograd.grad和hook来处理这种情况：**

In [103]:
# 利用autograd.grad获取中间节点梯度
torch.autograd.grad(d,c)
# 输出：(tensor([[1., 1., 1., 1.],
#        [1., 1., 1., 1.],
#        [1., 1., 1., 1.]]),)

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),)

#### 利用hook获取中间节点梯度

In [104]:

import torch
 
a = torch.ones(3,4,requires_grad=True)
b = torch.rand(3,4,requires_grad=True)
c = a*b
d = c.sum()
 
def print_grad(grad):
    print(grad)
 
# 给c注册hook
c_hook = c.register_hook(print_grad)
 
print(d.backward())
#输出：tensor([[1., 1., 1., 1.],
#        [1., 1., 1., 1.],
#        [1., 1., 1., 1.]])
 
#移除钩子
c_hook.remove()

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
None


# 7.损失函数

In [105]:
from torch import nn

help(nn)

Help on package torch.nn in torch:

NAME
    torch.nn

PACKAGE CONTENTS
    _VF
    _functions (package)
    _qscheme
    _reduction
    backends (package)
    cpp
    functional
    grad
    init
    intrinsic (package)
    modules (package)
    parallel (package)
    parameter
    qat (package)
    quantized (package)
    utils (package)

FILE
    /anaconda3/lib/python3.7/site-packages/torch/nn/__init__.py




In [141]:
import torch
import torch.nn as nn
import torch.nn.functional as F

sample = torch.ones(2, 2, requires_grad=True)
target = torch.Tensor([[0, 1], [2,3]])

In [142]:
type(sample), type(target)

(torch.Tensor, torch.Tensor)

In [143]:
sample.dtype, target.dtype

(torch.float32, torch.float32)

## 7.1 nn.L1Loss

$$
loss(x, y) = \frac{1}{N}\sum_{i=1}^{N}|x-y|
$$

取预测值和真实值绝对误差的平均数

In [144]:
criterion = nn.L1Loss()
loss = criterion(sample, target)
print(loss)

tensor(1., grad_fn=<L1LossBackward>)


## 7.2 nn.SmoothL1Loss

SmoothL1Loss也叫Huber Loss，误差在(-1, 1)上是平方损失，其他情况是L1损失

In [145]:
criterion = nn.SmoothL1Loss()
loss = criterion(sample, target)
loss

tensor(0.6250, grad_fn=<SmoothL1LossBackward>)

## 7.3 nn.MSELoss

$$
loss(x, y) = \frac{1}{N}\sum_{i=1}^{N}(x-y)^2
$$

平方损失函数，计算公式是预测值和真实值的差值的平方再取平均。

In [146]:
criterion = nn.MSELoss()
loss = criterion(sample, target)
loss

tensor(1.5000, grad_fn=<MseLossBackward>)

## 7.4 nn.BCELoss

$$
loss(o, t) = \frac{1}{N}\sum_{i=1}^{N}[t_i*log(o_i)+(1-t_i)*log(1-o_i)]
$$

二分类交叉熵损失函数

In [147]:
criterion = nn.BCELoss()
loss = criterion(sample, target)
loss

tensor(-13.8155, grad_fn=<BinaryCrossEntropyBackward>)

## 7.5 nn.CrossEntropyLoss

交叉熵损失函数

$$
loss(x, label) = -log\frac{e^{X_{label}}}{\sum_{j=1}^{N}e^{X_j}} = -x_{label} + log{\sum_{j=1}^{N}e^{X_j}}
$$


该损失函数结合了`nn.LogSoftmax()`和`nn.NLLLoss()`两个函数。它在做分类（具体几类）训练的时候是非常有用的。在训练过程中，对于每个类分配权值，可选的参数权值应该是一个1D张量。当你有一个不平衡的训练集时，这是是非常有用的。那么针对这个函数，下面将做详细的介绍。

### 7.5.1 什么是交叉熵？
交叉熵主要是用来判定实际的输出与期望的输出的接近程度，为什么这么说呢，举个例子：在做分类的训练的时候，如果一个样本属于第K类，那么这个类别所对应的的输出节点的输出值应该为1，而其他节点的输出都为0，即`[0,0,1,0,….0,0]`，这个数组也就是样本的Label，是神经网络最期望的输出结果。也就是说用它来衡量网络的输出与标签的差异，利用这种差异经过反向传播去更新网络参数。

### 7.5.2 交叉熵原理？
在说交叉熵之前，先说一下信息量与熵。

**信息量** ：它是用来衡量一个事件的不确定性的；一个事件发生的概率越大，不确定性越小，则它所携带的信息量就越小。假设X是一个离散型随机变量，其取值集合为X，概率分布函数为$p(x) = P(X=x), x\in X$ ，我们定义事件$X = x_0$ 的信息量为：$I(x_0) = -log(p(x_0))$ 当 $p(x_0)=1$ 时，熵将等于0，也就是说该事件的发生不会导致任何信息量的增加。

**熵**：它是用来衡量一个系统的混乱程度的，代表一个系统中信息量的总和；信息量总和越大，表明这个系统不确定性就越大。
计算公式为：
$$
H(x) = -\sum_{x_i}p(x_i)log(p(x_i))
$$

**交叉熵**：它主要刻画的是实际输出（概率）与期望输出（概率）的距离，也就是交叉熵的值越小，两个概率分布就越接近。假设概率分布p为期望输出，概率分布q为实际输出，$H(p, q)$ 为交叉熵，则
$$
H(p, q) = -\sum_{x}(p(x)log(q(x))+(1-p(x))log(1-q(x)))
$$

### 7.5.3 Pytorch中的CrossEntropyLoss()函数
实际上PyTorch中的CrossEntropyLoss的计算并不是采用的上式，而是：
$$
H(p, q) = -\sum_{x}p(x)log(q(x))
$$
它是交叉熵的另外一种形式。

Pytorch中CrossEntropyLoss()函数的主要是将softmax-log-NLLLoss合并到一块得到的结果。

    1、Softmax后的数值都在0~1之间，所以ln之后值域是负无穷到0。

    2、然后将Softmax之后的结果取log，将乘法改成加法减少计算量，同时保障函数的单调性 。

    3、NLLLoss的结果就是把上面的输出与Label对应的那个值拿出来(下面例子中就是：将log_output\logsoftmax_output中与y_target对应的值拿出来)，去掉负号，再求均值。 下面是我仿真写的一个例子：

In [162]:
import torch
import torch.nn as nn
x_input=torch.randn(3,3)#随机生成输入 
print('x_input:\n',x_input) 
y_target=torch.tensor([1,2,0])#设置输出具体值 print('y_target\n',y_target)

#计算输入softmax，此时可以看到每一行加到一起结果都是1
softmax_func=nn.Softmax(dim=1)
soft_output=softmax_func(x_input)
print('soft_output:\n',soft_output)

#在softmax的基础上取log
log_output=torch.log(soft_output)
print('log_output:\n',log_output)

#对比softmax与log的结合与nn.LogSoftmaxloss(负对数似然损失)的输出结果，发现两者是一致的。
logsoftmax_func=nn.LogSoftmax(dim=1)
logsoftmax_output=logsoftmax_func(x_input)
print('logsoftmax_output:\n',logsoftmax_output)

#pytorch中关于NLLLoss的默认参数配置为：reducetion=True、size_average=True
nllloss_func=nn.NLLLoss()
nlloss_output=nllloss_func(logsoftmax_output,y_target)
print('nlloss_output:\n',nlloss_output)

#直接使用pytorch中的loss_func=nn.CrossEntropyLoss()看与经过NLLLoss的计算是不是一样
crossentropyloss=nn.CrossEntropyLoss()
crossentropyloss_output=crossentropyloss(x_input,y_target)
print('crossentropyloss_output:\n',crossentropyloss_output)

x_input:
 tensor([[-0.1783,  1.3814,  0.4364],
        [ 0.8832,  0.4122, -0.8587],
        [-0.2509,  0.6555, -0.2324]])
soft_output:
 tensor([[0.1315, 0.6254, 0.2431],
        [0.5557, 0.3470, 0.0974],
        [0.2225, 0.5508, 0.2267]])
log_output:
 tensor([[-2.0290, -0.4693, -1.4143],
        [-0.5875, -1.0586, -2.3294],
        [-1.5028, -0.5964, -1.4843]])
logsoftmax_output:
 tensor([[-2.0290, -0.4693, -1.4143],
        [-0.5875, -1.0586, -2.3294],
        [-1.5028, -0.5964, -1.4843]])
nlloss_output:
 tensor(1.4338)
crossentropyloss_output:
 tensor(1.4338)


## 7.6 nn.NLLLoss
负对数似然(损失)函数 - Negative Log Likelihood Loss
$$
loss(x, label) = -x_{label}
$$

`nn.NLLLoss` 和 `nn.CrossEntropyLoss`的功能是非常相似的！通常用在多分类模型中。

- 示例见上

## 7.7 nn.NLLLoss2D

和上面类似，但是多了几个维度，一般用在图片上。
- input (N, C, H, W)
- target (N, H, W)
